In [1]:
import pandas as pd
import os
import numpy as np

from gym import Env
from gym.spaces import Discrete, Box
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
pd.options.display.max_columns= 200


2022-11-13 19:49:23.784840: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_files = os.listdir('data')
data_files.sort()
data_files

['.ipynb_checkpoints',
 'NFLBirthdays.csv',
 'RL_test_data.csv',
 'games.csv',
 'pffScoutingData.csv',
 'players.csv',
 'plays.csv',
 'processed',
 'raw',
 'train.csv',
 'week1.csv',
 'week2.csv',
 'week3.csv',
 'week4.csv',
 'week5.csv',
 'week6.csv',
 'week7.csv',
 'week8.csv']

In [3]:
games_df = pd.read_csv('./data/games.csv') # this file contains the schedule of games_df
plays_df = pd.read_csv('./data/RL_test_data.csv', index_col=0) # this file tells us about each play in each game
weeks_df = pd.read_csv('./data/week1.csv') # this file contains player tracking data

for file_name in data_files:
    
    if 'week' in file_name and file_name != 'week1.csv':
    
        d = pd.read_csv(f"./data/{file_name}", engine='python', error_bad_lines=False)

        weeks_df = pd.concat([d,weeks_df])



/var/folders/s7/3dcc3pkd0kb7tgrb1sc5zlzh0000gn/T/ipykernel_5582/377191410.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  d = pd.read_csv(f"./data/{file_name}", engine='python', error_bad_lines=False)
/var/folders/s7/3dcc3pkd0kb7tgrb1sc5zlzh0000gn/T/ipykernel_5582/377191410.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  d = pd.read_csv(f"./data/{file_name}", engine='python', error_bad_lines=False)
Skipping line 1093237: unexpected end of data
/var/folders/s7/3dcc3pkd0kb7tgrb1sc5zlzh0000gn/T/ipykernel_5582/377191410.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  d = pd.read_csv(f"./data/{file_name}", engine='python', error_bad_lines=False)
Skipping line 908085: unexpected end of data
/var/fold

In [4]:
class footballEnv(Env):
    def __init__(self, df):
        
        # idx used to start on the first series
        self.next_series = 0

        # get an array of all available sereies
        self.unique_series = df.series_id.unique()        

        # measure number of series ids
        self.total_series = self.unique_series.shape[0]-1
                        
        # store this series df 
        self.data_frame = df
            
        # actions to take: left, right, middle, deep
        self.action_space = Discrete(4)
                        
        # length of field
        self.observation_space = Box(low=-20, high=120, shape=(9,))
                
    def step(self, action):
        info = {}
        done=False
        reward = 0
        
        if self.current_play == self.series_df_length:
            done=True
        else:
            self.current_play += 1

            df = self.series_df

            play_in_series = df.loc[self.current_play]

            yardage_gain = play_in_series.prePenaltyPlayResult

            self.state = play_in_series.absoluteYardlineNumber
            self.quarter = play_in_series.quarter
            self.down = play_in_series.down
            self.yards_to_go = play_in_series.yardsToGo
            self.play_direction_right = play_in_series.play_direction_is_right
            self.home_team_score = play_in_series.preSnapHomeScore
            self.visitor_team_score = play_in_series.preSnapVisitorScore
            self.game_clock_seconds = play_in_series.game_clock_seconds
                        
            # should be using yardage gain proportional to field size
            reward = yardage_gain                        
                
        return np.array([self.state, self.quarter, self.down, self.yards_to_go, self.play_direction_right, self.home_team_score, self.visitor_team_score, self.game_clock_seconds, self.current_play]), reward, done, info 
            
    def render(self):
        pass
        
    def reset(self):#, plays_remaining):        
        
        # get first series_id
        unique_series = self.unique_series

        # get first series_id
        series_id = unique_series[self.next_series]
        
        self.next_series = random.randint(1,self.total_series)
        
        df = self.data_frame
        # measure number of plays in this series
        self.n_plays_in_series = df.shape[0]-1
        
        # get all data for current series
        df = df[df.series_id==series_id].reset_index()
            
        self.series_df = df
        self.series_df_length = df.shape[0]-1
                
        self.current_play = 0
        self.state = df.absoluteYardlineNumber[self.current_play]
        self.quarter = df.quarter[self.current_play]
        self.down = df.down[self.current_play]
        self.yards_to_go = df.yardsToGo[self.current_play]
        self.play_direction_right = df.play_direction_is_right[self.current_play]
        self.home_team_score = df.preSnapHomeScore[self.current_play]
        self.visitor_team_score = df.preSnapVisitorScore[self.current_play]
        self.game_clock_seconds = df.game_clock_seconds[self.current_play]

        return np.array([self.state, self.quarter, self.down, self.yards_to_go, self.play_direction_right, self.home_team_score, self.visitor_team_score, self.game_clock_seconds, self.current_play])
    

In [5]:
env = footballEnv(plays_df)


In [6]:
env.observation_space.sample()


array([77.706184 , -4.3702993, 59.233276 , 17.967237 , -2.4887102,
       26.309713 , 10.355825 , 13.045078 , -9.055427 ], dtype=float32)

In [7]:
# # games_df we'll train on
# episodes = plays_df.series_id.unique()

# for episode in episodes:

#     game = plays_df[plays_df.gameId==episode].sort_values(['quarter', 'down', 'game_clock_seconds'], ascending=[True, True, False])
    
#     game = game.reset_index()
               
#     state = env.reset()
#     done = False
#     score = 0 
    
#     while not done:
#         #env.render()
#         action = env.action_space.sample()
#         n_state, reward, done, info = env.step(action)
#         score+=reward
#     print('Episode:{} Score:{}'.format(episode, score))

In [8]:
states = env.observation_space.shape
states

(9,)

In [9]:
actions = env.action_space.n
actions

4

In [10]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=(1,9)))
    model.add(Dropout(.5))
    model.add(Dense(24, activation='relu'))
    model.add(Dropout(.5))
    model.add(Flatten())
    model.add(Dense(actions, activation='linear'))
    
    return model


In [11]:
try:
    del model
except:
    pass

In [12]:
model = build_model(states, actions)


In [13]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 24)             240       
                                                                 
 dropout (Dropout)           (None, 1, 24)             0         
                                                                 
 dense_1 (Dense)             (None, 1, 24)             600       
                                                                 
 dropout_1 (Dropout)         (None, 1, 24)             0         
                                                                 
 flatten (Flatten)           (None, 24)                0         
                                                                 
 dense_2 (Dense)             (None, 4)                 100       
                                                                 
Total params: 940
Trainable params: 940
Non-trainable pa

In [14]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [15]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

/Users/ciannudi-muldoon/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2022-11-13 19:50:48.531746: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-13 19:50:48.541831: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 22:31 - reward: 0.0000e+00

/Users/ciannudi-muldoon/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/Users/ciannudi-muldoon/opt/anaconda3/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/Users/ciannudi-muldoon/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   41/10000 [..............................] - ETA: 4:07 - reward: 2.8049

/Users/ciannudi-muldoon/opt/anaconda3/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/Users/ciannudi-muldoon/opt/anaconda3/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/Users/ciannudi-muldoon/opt/anaconda3/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid

10000/10000 [==============================] - 80s 8ms/step - reward: 4.6677
2575 episodes - episode_reward: 18.116 [-16.000, 169.000] - loss: 130.490 - mae: 8.526 - mean_q: 10.506

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 85s 8ms/step - reward: 4.9344
2508 episodes - episode_reward: 19.679 [-21.000, 169.000] - loss: 70.378 - mae: 11.903 - mean_q: 13.440

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: 4.5575
2554 episodes - episode_reward: 17.830 [-27.000, 169.000] - loss: 74.120 - mae: 12.946 - mean_q: 14.662

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 79s 8ms/step - reward: 4.6635
2594 episodes - episode_reward: 17.998 [-27.000, 165.000] - loss: 72.480 - mae: 12.803 - mean_q: 14.467

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 82s 8ms/step - reward: 4.6699
done, took 403.670 seconds
